In [1]:
import numpy as np
import torch
import torch.nn as nn

from datasets.posetrack21 import PoseTrack21
from utilities.utilities import keypoints_to_mask, accuracy, show_image, HiddenPrints, show_heatmaps, keypoints_to_label_flag
from mmpose.evaluation import pose_pck_accuracy, keypoint_pck_accuracy
from mmpose.codecs import UDPHeatmap
from models.backbone.vit_with_fusion import FusionVit
from models.head.deconv import DeconvHead
from models.PoseEstimate import PoseEstimate
from models.head import MaskHead, ResMaskHead

In [2]:
# device = "cpu"
device = "cuda"
pretrained_path = "./checkpoints/vitpose_base_coco_aic_mpii.pth"
encoder = FusionVit(pretrained_path=pretrained_path)
neck = nn.Identity()
head = MaskHead(pretrained_path=pretrained_path)
model = PoseEstimate(encoder=encoder, neck=neck, head=head)
model.to(device)
model_path = "./checkpoints/Fusion_Mask_01.pth"
model_dict = torch.load(model_path, map_location=device)
model.load_state_dict(model_dict["model_state_dict"])
epoch = model_dict["epoch"]
print(f"Loaded model from {model_path} at epoch {epoch}")
with HiddenPrints():
    model.to(device)
    model.eval()

Loaded model from ./checkpoints/Fusion_Mask_01.pth at epoch 2200


In [3]:
def normalize_heatmaps(heatmaps):
    """
    Normalizes heatmaps to the range [0, 1].

    Parameters:
    - heatmaps: torch.Tensor of shape (num_joints, height, width)
                17 * 64 * 48 in your case.

    Returns:
    - normalized_heatmaps: torch.Tensor of shape (num_joints, height, width)
    """
    num_joints, height, width = heatmaps.shape
    normalized_heatmaps = torch.zeros_like(heatmaps, dtype=torch.float32)

    for i in range(num_joints):
        min_val = torch.min(heatmaps[i])
        max_val = torch.max(heatmaps[i])

        if max_val != min_val:
            normalized_heatmaps[i] = (heatmaps[i] - min_val) / (max_val - min_val)

    return normalized_heatmaps

# Example usage
heatmaps = torch.rand(17, 64, 48)  # Generating random heatmaps for 17 joints, 64x48 in size.
normalized_heatmaps = normalize_heatmaps(heatmaps)

# Now `normalized_heatmaps` contains the normalized values in the range [0, 1].


In [4]:
dataset_root_dir = "/home/junfeng/datasets/PoseTrack21"
dataset_train = PoseTrack21(
    root_dir=dataset_root_dir,
    set="train",
)

dataset_test = PoseTrack21(
    root_dir=dataset_root_dir,
    set="test",
)

print(len(dataset_train))
print(len(dataset_test))

593
170


In [6]:
def ReLu(x):
    return np.maximum(x, 0)

def toP(x):
    if x < 0:
        return 0
    else:
        return 1
    
def get_mean_average_acc(TP, P):
    return sum(TP) / sum(P)

In [19]:
dataset = dataset_test
loss_val = 0
mean_avg_acc = 0
mean_avg_acc_batch = 0
TP = [0]*17
P = [0]*17
count = 0
thr = 0.05
batch_size = 1

# collate
for i in range(1000):
    mask_list = []
    heatmaps_list = []
    pred_heatmaps_list = []
    for j in range(batch_size):
        video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[np.random.randint(
            0, len(dataset))]
        heatmaps = heatmaps.unsqueeze(0).numpy()
        mask = keypoints_to_mask(keypoints)
        with torch.no_grad():
            video_transformed = video_transformed.to(device).unsqueeze(0)
            pred_heatmaps = model.predict(video_transformed)
            pred_heatmaps = pred_heatmaps.cpu().numpy()
            acc, avg_acc, _ = pose_pck_accuracy(
                pred_heatmaps, heatmaps, mask, thr=thr)
            mean_avg_acc += avg_acc
            count += 1
            P = [P[k] + toP(acc[k]) for k in range(len(acc))]
            TP = [TP[k] + ReLu(acc[k]) for k in range(len(acc))]
            mask_list.append(np.squeeze(mask))
            heatmaps_list.append(np.squeeze(heatmaps))
            pred_heatmaps_list.append(np.squeeze(pred_heatmaps))
    mask_list = np.array(mask_list)
    heatmaps_list = np.array(heatmaps_list)
    pred_heatmaps_list = np.array(pred_heatmaps_list)
    _, avg_acc_batch, _ = pose_pck_accuracy(
        pred_heatmaps_list, heatmaps_list, mask_list, thr=thr)
    mean_avg_acc_batch += avg_acc_batch
    print(f"i: {i}, mean_avg_acc: {mean_avg_acc / count}, mean_avg_acc_batch: {mean_avg_acc_batch / (i+1)}, mean_avg_acc_TP: {get_mean_average_acc(TP, P)}", end="\r")

In [41]:
dataset = dataset_test
loss_val = 0
mean_avg_acc = 0
mean_avg_acc_batch = 0
avg_acc_list = []
weight_list = []
TP = [0]*17
P = [0]*17
count = 0
thr = 0.05
batch_size = 4

# collate
for i in range(1):
    mask_list = []
    heatmaps_list = []
    pred_heatmaps_list = []
    for j in range(batch_size):
        video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[np.random.randint(
            0, len(dataset))]
        heatmaps = heatmaps.unsqueeze(0).numpy()
        mask = keypoints_to_mask(keypoints)
        with torch.no_grad():
            video_transformed = video_transformed.to(device).unsqueeze(0)
            pred_heatmaps = model.predict(video_transformed)
            pred_heatmaps = pred_heatmaps.cpu().numpy()
            acc, avg_acc, cnt = pose_pck_accuracy(
                pred_heatmaps, heatmaps, mask, thr=thr)
            mean_avg_acc += avg_acc
            avg_acc_list.append(avg_acc)
            weight_list.append(cnt)
            count += 1
            P = [P[k] + toP(acc[k]) for k in range(len(acc))]
            TP = [TP[k] + ReLu(acc[k]) for k in range(len(acc))]
            mask_list.append(np.squeeze(mask))
            heatmaps_list.append(np.squeeze(heatmaps))
            pred_heatmaps_list.append(np.squeeze(pred_heatmaps))
    mask_list = np.array(mask_list)
    heatmaps_list = np.array(heatmaps_list)
    pred_heatmaps_list = np.array(pred_heatmaps_list)
    _, avg_acc_batch, cnt_batch = pose_pck_accuracy(
        pred_heatmaps_list, heatmaps_list, mask_list, thr=thr)
    print(cnt_batch)
    mean_avg_acc_batch += avg_acc_batch
    print(f"i: {i}, mean_avg_acc: {mean_avg_acc / count}, mean_avg_acc_batch: {mean_avg_acc_batch / (i+1)}, mean_avg_acc_TP: {get_mean_average_acc(TP, P)}")
    print(f"weighted_average: {weighted_average(avg_acc_list, weight_list)}")

14
i: 0, mean_avg_acc: 0.6273809523809524, mean_avg_acc_batch: 0.5833333333333333, mean_avg_acc_TP: 0.6060606060606061
weighted_average: 0.6060606060606061


In [17]:
dataset = dataset_test
loss_val = 0
mean_avg_acc = 0
mean_avg_acc_batch = 0
TP = [0]*17
P = [0]*17
count = 0
thr = 0.05
batch_size = 1

# collate
for i in range(1000):
    mask_list = []
    heatmaps_list = []
    pred_heatmaps_list = []
    for j in range(batch_size):
        video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[np.random.randint(
            0, len(dataset))]
        heatmaps = heatmaps.unsqueeze(0).numpy()
        mask = keypoints_to_mask(keypoints)
        with torch.no_grad():
            video_transformed = video_transformed.to(device).unsqueeze(0)
            pred_heatmaps = model.predict(video_transformed)
            pred_heatmaps = pred_heatmaps.cpu().numpy()
            acc, avg_acc, _ = accuracy(
                pred_heatmaps, heatmaps)
            acc = acc[1:]
            mean_avg_acc += avg_acc
            count += 1
            P = [P[k] + toP(acc[k]) for k in range(len(acc))]
            TP = [TP[k] + ReLu(acc[k]) for k in range(len(acc))]
            mask_list.append(np.squeeze(mask))
            heatmaps_list.append(np.squeeze(heatmaps))
            pred_heatmaps_list.append(np.squeeze(pred_heatmaps))
    mask_list = np.array(mask_list)
    heatmaps_list = np.array(heatmaps_list)
    pred_heatmaps_list = np.array(pred_heatmaps_list)
    _, avg_acc_batch, _ = accuracy(
        pred_heatmaps_list, heatmaps_list)
    mean_avg_acc_batch += avg_acc_batch
    print(f"i: {i}, mean_avg_acc: {mean_avg_acc / count}, mean_avg_acc_batch: {mean_avg_acc_batch / (i+1)}, mean_avg_acc_TP: {get_mean_average_acc(TP, P)}", end="\r")

In [18]:
dataset = dataset_test
loss_val = 0
mean_avg_acc = 0
mean_avg_acc_batch = 0
TP = [0]*17
P = [0]*17
count = 0
thr = 0.05
batch_size = 4

# collate
for i in range(1000):
    mask_list = []
    heatmaps_list = []
    pred_heatmaps_list = []
    for j in range(batch_size):
        video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[np.random.randint(
            0, len(dataset))]
        heatmaps = heatmaps.unsqueeze(0).numpy()
        mask = keypoints_to_mask(keypoints)
        with torch.no_grad():
            video_transformed = video_transformed.to(device).unsqueeze(0)
            pred_heatmaps = model.predict(video_transformed)
            pred_heatmaps = pred_heatmaps.cpu().numpy()
            acc, avg_acc, _ = accuracy(
                pred_heatmaps, heatmaps)
            acc = acc[1:]
            mean_avg_acc += avg_acc
            count += 1
            P = [P[k] + toP(acc[k]) for k in range(len(acc))]
            TP = [TP[k] + ReLu(acc[k]) for k in range(len(acc))]
            mask_list.append(np.squeeze(mask))
            heatmaps_list.append(np.squeeze(heatmaps))
            pred_heatmaps_list.append(np.squeeze(pred_heatmaps))
    mask_list = np.array(mask_list)
    heatmaps_list = np.array(heatmaps_list)
    pred_heatmaps_list = np.array(pred_heatmaps_list)
    _, avg_acc_batch, _ = accuracy(
        pred_heatmaps_list, heatmaps_list)
    mean_avg_acc_batch += avg_acc_batch
    print(f"i: {i}, mean_avg_acc: {mean_avg_acc / count}, mean_avg_acc_batch: {mean_avg_acc_batch / (i+1)}, mean_avg_acc_TP: {get_mean_average_acc(TP, P)}", end="\r")

KeyboardInterrupt: 

In [7]:
dataset = dataset_test
loss_val = 0
mean_avg_acc = 0
mean_avg_acc_udp = 0
TP = [0]*17
P = [0]*17
mean_acc = [0]*17
mean_acc_upd = [0]*17
TP_udp = [0]*17
P_udp = [0]*17
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        acc, avg_acc, cnt = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        P = [P[i] + toP(acc[i]) for i in range(len(acc))]
        TP = [TP[i] + ReLu(acc[i]) for i in range(len(acc))]
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        acc_udp, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        P_udp = [P_udp[i] + toP(acc_udp[i]) for i in range(len(acc_udp))]
        TP_udp = [TP_udp[i] + ReLu(acc_udp[i]) for i in range(len(acc_udp))]
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, {get_mean_average_acc(TP, P)} {get_mean_average_acc(TP_udp, P_udp)},", end="\r")

In [8]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [9]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed, 4)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [5]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [6]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed, 4)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [6]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed, 8)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [5]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed, 4)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [6]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 100)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [7]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 100)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [7]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 200)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

KeyboardInterrupt: 

In [8]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 100)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 100)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [ ]:
dataset = dataset_test
loss_val = 0
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
while count < 5000:
    count += 1
    i = np.random.randint(0, len(dataset))
    video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 200)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [10]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [11]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed, 4)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [7]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.unsqueeze(0).cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed, 4)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 4, 200)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [21]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 10)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [17]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [18]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model.denoise(video_transformed, pred_heatmaps, 200)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [19]:
dataset = dataset_test
mean_avg_acc_udp = 0
mean_avg_acc = 0
count = 0
thr = 0.05
for video, video_transformed, expanded_bbox, keypoints, keypoints_transformed, heatmaps, image_id, track_id in dataset.get_for_eval(1):
    count += 1
    mask = keypoints_to_mask(keypoints)
    image = video[1]
    decoder = UDPHeatmap(input_size=(192, 256), heatmap_size=(48, 64))
    image_transformed = video_transformed[1]
    video_transformed = video_transformed.to(device).unsqueeze(0)
    heatmaps = heatmaps.cpu().numpy()
    with torch.no_grad():
        pred_heatmaps = model.predict(video_transformed)
        pred_heatmaps = normalize_heatmaps(pred_heatmaps.squeeze()).unsqueeze(0)
        pred_heatmaps = model(video_transformed, pred_heatmaps)
        pred_heatmaps = pred_heatmaps.cpu().numpy()
        _, avg_acc, _ = pose_pck_accuracy(
            pred_heatmaps, heatmaps, mask, thr=thr)
        mean_avg_acc += avg_acc
        pred_heatmaps = pred_heatmaps.squeeze()
        pred_keypoints = decoder.decode(pred_heatmaps)[0]
        normalize = np.tile(np.array([[256, 192]]), (1, 1))
        _, avg_acc_udp, _ = keypoint_pck_accuracy(
            pred=pred_keypoints, gt=keypoints_transformed[:, :2].numpy(), mask=mask, thr=thr, norm_factor=normalize)
        mean_avg_acc_udp += avg_acc_udp
        print(
            f"count: {count}, mean_avg_acc_udp: {mean_avg_acc_udp / count}, mean_avg_acc {mean_avg_acc / count}", end="\r")

In [ ]:
dataset = dataset_test
loss_val = 0
acc = 0
count = 0
i = np.random.randint(0, len(dataset))
video, video_transformed, keypoints, keypoints_transformed, heatmaps = dataset[i]
image = video[1]
H, W, _ = image.shape
decoder = UDPHeatmap(input_size=(W, H), heatmap_size=(48, 64))
image_transformed = video_transformed[1]
video_transformed = video_transformed.to(device).unsqueeze(0)
show_image(image, keypoints)
with torch.no_grad():
    pred_heatmaps = model.predict(video_transformed, 8)
    show_image(image, heatmaps=pred_heatmaps.squeeze(0).cpu().numpy())
    show_heatmaps(pred_heatmaps.squeeze(0).cpu().numpy())
    